<div align="center">

###### ***Assignment 1: by Jonaz Juan Sayson and Fabiola Villanueva***

### **"<u>Balanced Risk Set Matching</u>"**
by Yunfei Paul Li, Kathleen J. Propert, and Paul R. Rosenbaum

</div>

<div style="text-align: justify;">

##### **Introduction**

The journal introduces a new matching method that is applied in an observational study to assess a treatment, cystoscopy and hydrodistension - **to create balanced comparison groups when random assignment is not possible**.

*The article "Balanced Risk Set Matching" was published in the Journal of the American Statistical Association (JASA). It is a peer-reviewed journal that publishes articles on statistical theory and methods, applications, and data science. The journal has both print (ISSN: 0162-1459) and online (ISSN: 1537-274X) versions. The JASA's focus is on original and important contributions to statistical methods, practice, and theory.*

##### **Key Concept**

The key idea for BRSM is that when **patient A is given treatment at a certain time *t*, patient A is matched to patient B with similar history of symptoms up to time *t*** - wherein patient B has not received the treatment up until time *t*.

##### **Methodology**

A treatment is studied for a chronic disease called **Interstitial Cystitis**. The treatment is called **cystoscopy and hydrodistention**, and it’s <u>given to patients based on their symptoms</u>. To understand if the treatment works, we compare patients who received it to similar patients who didn’t.

1. **Risk Set Matching**
	- When a patient gets treatment at a specific time, we find another patient who had similar symptoms up to that point but didn't get the treatment; **finding treated and untreated patients with similar symptoms** - matching patients.
2. **Balancing Symptoms**
	- How do we exactly know that the untreated and treated patients have **similar symptom patterns?** - **integer programming**.
3. **Choosing the Best Matches**
	- Treated and untreated patients with similar symptom patterns are then narrowed down further by **selecting a group based off of similarity in attributes**.
4. **Checking for Hidden Biases**
	- Results are then tested in the matter of sensitivity from hidden factors that could affect on who gets the treatment. **A standard way to check for these biases must be shown w/o biases** that shall suit best for this study.

> <u>***Key Words for Query***</u>
>
> Integer programming? Matched sampling? Network flow? Observational study? Sensitivity analysis?

Patients with similar symptom patterns (using **risk set matching**) are grouped into "treated" (those who received the treatment) and "untreated" (those who didn’t) - take note of the time these patients are diagnosed. To make the groups as similar as possible, patients are further matched based on their characteristics (like age, health history, etc.) using a math technique called **integer programming**. The results are tested to ensure they’re not skewed by hidden biases, making the findings more trustworthy.

</div>

---
<div align="center">

### **"Matching to Achieve Comparable Histories"**

##### **Observational Study of <u>Interstitial Cystitis</u>**

</div>

<div style="text-align: justify;">

This study **looks at how a surgical procedure called cystoscopy and hydrodistension affects the symptoms of Interstitial Cystitis** (IC), a chronic condition causing bladder pain and frequent, urgent urination. Unlike a urinary tract infection, **IC has no evidence of infection, and its causes and progression are not well understood**.

In an ideal scenario, the effectiveness of a treatment would be tested in a randomized controlled trial, where **patients are randomly assigned to either receive the treatment or not**. This ensures that the groups being compared are similar before treatment. However, in real-world settings, especially with chronic diseases like IC, randomized trials aren't always possible due to ethical or practical reasons. Instead, **treatments are often given based on the severity of symptoms**, which can **make it harder to compare treated and untreated patients fairly**.

This study uses data from the Interstitial Cystitis Data Base (ICDB), which tracked patients with IC symptoms (like urinary urgency, frequency, and pelvic pain) for up to 4 years. Patients were evaluated every 3 months, and three key symptoms were measured: **pain, urgency, and nighttime urination frequency**. Pain and urgency were rated on a scale from 0 to 9, with higher numbers indicating worse symptoms.

Patients in the database were treated according to standard clinical practices, and some underwent cystoscopy and hydrodistension, likely due to severe or worsening symptoms. The study aims to understand how this surgical intervention impacts IC symptoms over time, even though the treatment wasn't randomly assigned.

> <u>***What are the procedures involved in the journal?***</u>
> 
>**Balanced Risk Set Matching** is **performed using a new algorithm to achieve similar patients** in each pair and balanced groups in aggregate.
>
>**Optimal balanced matching** is conducted **to minimize the total distance within matched pairs**. This involves finding a balanced pair matching M that minimizes the total distance across all balanced pair matchings. This can be achieved through integer programming methods.
> 
>**Minimum cost flow in a network** is used for **optimal pair matching**.
>
>**Sensitivity analysis** is conducted **to assess how hidden biases might alter the conclusions** of the observational study. This involves a parameter that describes the magnitude of departure from a randomised experiment.
>
>**Permutation inference** is used, which **involves risk set matching to justify simple, conventional permutation inferences** under a general model for the decision to apply the treatment at a particular moment in response to time-varying symptoms.
>
>**Integer programming** is used, where **the optimal balanced matching problem is expressed as minimizing a linear function of integer variables**, subject to linear equality and inequality constraints. The balance conditions are treated as a penalty in the objective function.

</div>